## Connecting to Amazon Keyspaces using server-side credentials 

This code shows how to connect to Amazon Keyspaces from SageMaker using an [service-specific credentials](https://docs.aws.amazon.com/keyspaces/latest/devguide/programmatic.credentials.html) for an existing AWS Identity and Access Management (IAM) user.


Before we start we need to generate the Keyspaces credential and use SecretManager to securly store credentials. 


1. Generate [Keyspaces Service-Specific Credentials](https://docs.aws.amazon.com/keyspaces/latest/devguide/programmatic.credentials.html)


The following is an example of a service-specific credential .

```
"ServiceSpecificCredential": {
        "CreateDate": "2019-10-09T16:12:04Z",
        "ServiceName": "cassandra.amazonaws.com",
        "ServiceUserName": "keyspace-user1-at-11122223333",
        "ServicePassword": "wJalrXUtnFEMI/K7MDENG/bPxRfiCYEXAMPLEKEY",
        "ServiceSpecificCredentialId": "ACCAYFI33SINPGJEBYESF",
        "UserName": " keyspace-user1",
        "Status": "Active"
    }
}
```

2. Store ServiceUserName and ServicePassword in the SecretManager.  As a best practice we don't want to store credential in SageMaker Notebooks in plain text. 

  In this example I'm using 
_Keyspaces_Server_Generated_credential_ as a Secret name and _keyspaces_generated_id_ and _keyspaces_generated_pw_ fields to store Keyspaces ID and password. 


In [ ]:
#Import 
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2 , CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider

ssl_context = SSLContext(PROTOCOL_TLSv1_2 )

In [ ]:
#Download certificate 
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O 

In [ ]:
# Load certificate 
ssl_context.load_verify_locations('sf-class2-root.crt') 

In [ ]:
#Getting Credential from Secert Manager 
#Need to have a SecretsManagerReadWrite Policy

import boto3
import base64
from botocore.exceptions import ClientError
import json

secret_name = "Keyspaces_Server_Generated_credential"
region_name = "us-west-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)


get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

# Decrypts secret using the associated KMS CMK.
# Depending on whether the secret is a string or binary, one of these fields will be populated.
if 'SecretString' in get_secret_value_response:
    secret = json.loads(get_secret_value_response['SecretString'])
    ServiceUserName = secret['keyspaces_generated_id']
    ServicePassword = secret['keyspaces_generated_pw']
    print("id:",ServiceUserName, ",  pw:",ServicePassword)
else:
    decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
             
            

In [ ]:
auth_provider = PlainTextAuthProvider(username=ServiceUserName, password=ServicePassword)

In [ ]:
cluster = Cluster(['cassandra.us-west-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)


In [ ]:
session = cluster.connect()
cql = "select * from  system_schema.keyspaces "
r = session.execute(cql)
print(r.current_rows)

In [ ]:
# Load rows to Panda DataFrame 
from pandas import DataFrame

df = DataFrame(r)
print(df)